Low Risk Investment

In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

**Dates**

In [2]:
# Define path to files
# For MacOS
PATH = "/Users/vanessalieberg/code/Megeve2022/Finance_python/data_stocks/"

# Start date defaults
S_YEAR = 2019
S_MONTH = 1
S_DAY = 3
S_DATE_STR = "2019-01-03"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2022
E_MONTH = 4
E_DAY = 1
E_DATE_STR = "2022-04-01"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

**Get Stock File Names in a List**

In [3]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

# On MacOS Only remove  '.DS_Store', CRC GRUB AAN ARNC
#tickers.remove('.DS_Store')

tickers.sort()
len(tickers)

2895

**Function that Returns a Dataframe from a CSV**

In [4]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv(ticker):
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(PATH + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

**Function that Saves Dataframe to CSV**

In [5]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

**Return 1st Valid Date in Dataframe**

In [6]:
# Receives a start and end date and returns the 1st date in that range
def get_valid_dates(df, sdate, edate):
    
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate) 
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
    
        # Get smallest date that matches
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
    
        # Add leading zeros to date
        # Split string on - and if a number is less than 2 add a zero in front
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
        print(date_leading, " ", date_ending)
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending

**Returns Return on Investment over Time**

In [7]:
# Get ROI between 2 dates
def roi_between_dates(df, sdate, edate):
    
    try:
        # Gets the Adj Close price for 1st & last date 
        start_val = df.loc[sdate,'Adj Close'] 
        end_val = df.loc[edate,'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi

**Get Mean Between Dates**

In [8]:
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].mean()

**Get Standard Deviation Between Dates**

In [9]:
def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].std()

**Get Coefficient of Variation Between Dates**

In [10]:
def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    return sd / mean

**Test Functions**

In [11]:
# Get our 1st ticker
tickers[0]
print("Dataframe for Ticker", tickers[0])

Dataframe for Ticker A


In [12]:
# Get a dataframe for that ticker
stock_a = get_df_from_csv(tickers[0])
stock_a

Unnamed: 0        Date   Adj Close  daily_return
0             0  2019-01-02   64.161537           NaN
1             1  2019-01-03   61.797855     -0.036840
2             2  2019-01-04   63.936897      0.034614
3             3  2019-01-07   65.294556      0.021234
4             4  2019-01-08   66.251747      0.014660
..          ...         ...         ...           ...
815         815  2022-03-28  135.207718     -0.002137
816         816  2022-03-29  138.203018      0.022153
817         817  2022-03-30  135.247665     -0.021384
818         818  2022-03-31  132.122574     -0.023106
819         819  2022-04-01  133.759995      0.012393

[820 rows x 4 columns]

In [13]:
# Get the first and last valid date in the date range
print(get_valid_dates(stock_a, '2020-01-01', '2020-12-31'))
sdate, edate = get_valid_dates(stock_a, '2020-01-01', '2020-12-31')
sdate
edate

2020-01-02   2020-12-31
('2020-01-02', '2020-12-31')
2020-01-02   2020-12-31


'2020-12-31'

In [14]:
print("Adj Close Mean :", get_mean_between_dates(stock_a, sdate, edate))
print("Adj Close SD :", get_sd_between_dates(stock_a, sdate, edate))
print("Adj Close COV :", get_cov_between_dates(stock_a, sdate, edate))


Adj Close Mean : 91.50280080522809
Adj Close SD : 13.753514017505603
Adj Close COV : 0.1503070277245523


In [15]:
stock_a = stock_a.set_index(['Date'])
stock_a

Unnamed: 0   Adj Close  daily_return
Date                                            
2019-01-02           0   64.161537           NaN
2019-01-03           1   61.797855     -0.036840
2019-01-04           2   63.936897      0.034614
2019-01-07           3   65.294556      0.021234
2019-01-08           4   66.251747      0.014660
...                ...         ...           ...
2022-03-28         815  135.207718     -0.002137
2022-03-29         816  138.203018      0.022153
2022-03-30         817  135.247665     -0.021384
2022-03-31         818  132.122574     -0.023106
2022-04-01         819  133.759995      0.012393

[820 rows x 3 columns]

In [16]:
stock_a.drop(columns = 'Unnamed: 0', inplace=True)
stock_a

Adj Close  daily_return
Date                                
2019-01-02   64.161537           NaN
2019-01-03   61.797855     -0.036840
2019-01-04   63.936897      0.034614
2019-01-07   65.294556      0.021234
2019-01-08   66.251747      0.014660
...                ...           ...
2022-03-28  135.207718     -0.002137
2022-03-29  138.203018      0.022153
2022-03-30  135.247665     -0.021384
2022-03-31  132.122574     -0.023106
2022-04-01  133.759995      0.012393

[820 rows x 2 columns]

In [17]:
print("Return on Investment since 2020:", roi_between_dates(stock_a, sdate, edate))

Return on Investment since 2020: 0.38750012502884673


**COV & ROI for All Stocks Over Defined Period**

In [18]:
def get_cov_ror(tickers, sdate, edate):
    # Define column names for dataframe
    col_names = ["Ticker", "COV", "ROI"]
    
    # Create dataframe with column names
    df = pd.DataFrame(columns = col_names)
    
    for ticker in tickers:
        print("Working on :", ticker)
        s_df = get_df_from_csv(ticker)
    
        sdate2, edate2 = get_valid_dates(s_df, sdate, edate)
    
        cov = get_cov_between_dates(s_df, sdate2, edate2)
    
        # Set date as index
        s_df = s_df.set_index(['Date'])
        roi = roi_between_dates(s_df, sdate2, edate2)

        # Add stock data to new dataframe row
        # len provides the length of the dataframe which is the next open index
        df.loc[len(df.index)] = [ticker, cov, roi]
    
    return df

In [20]:
# Remove CRC GRUB AAN ARNC
tickers.remove('BEAT')
tickers.remove('CHK')
tickers.remove('GPOR')
tickers.remove('NE')
tickers.remove('RESI')
tickers.remove('SMRT')
tickers.remove('VAL')



market_df = get_cov_ror(tickers, '2020-01-03', '2020-12-31')

Working on : A
2020-01-06   2020-12-31
Working on : AA
2020-01-06   2020-12-31
Working on : AAL
2020-01-06   2020-12-31
Working on : AAME
2020-01-06   2020-12-31
Working on : AAN
2020-11-25   2020-12-31
Working on : AAOI
2020-01-06   2020-12-31
Working on : AAON
2020-01-06   2020-12-31
Working on : AAP
2020-01-06   2020-12-31
Working on : AAPL
2020-01-06   2020-12-31
Working on : AAT
2020-01-06   2020-12-31
Working on : AAWW
2020-01-06   2020-12-31
Working on : ABBV
2020-01-06   2020-12-31
Working on : ABC
2020-01-06   2020-12-31
Working on : ABCB
2020-01-06   2020-12-31
Working on : ABEO
2020-01-06   2020-12-31
Working on : ABG
2020-01-06   2020-12-31
Working on : ABIO
2020-01-06   2020-12-31
Working on : ABM
2020-01-06   2020-12-31
Working on : ABMD
2020-01-06   2020-12-31
Working on : ABR
2020-01-06   2020-12-31
Working on : ABT
2020-01-06   2020-12-31
Working on : ABTX
2020-01-06   2020-12-31
Working on : AC
2020-01-06   2020-12-31
Working on : ACA
2020-01-06   2020-12-31
Working o

2020-01-06   2020-12-31
Working on : ASH
2020-01-06   2020-12-31
Working on : ASIX
2020-01-06   2020-12-31
Working on : ASMB
2020-01-06   2020-12-31
Working on : ASPN
2020-01-06   2020-12-31
Working on : ASPS
2020-01-06   2020-12-31
Working on : ASRT
2020-01-06   2020-12-31
Working on : ASRV
2020-01-06   2020-12-31
Working on : ASTC
2020-01-06   2020-12-31
Working on : ASTE
2020-01-06   2020-12-31
Working on : ASUR
2020-01-06   2020-12-31
Working on : ASYS
2020-01-06   2020-12-31
Working on : ATEC
2020-01-06   2020-12-31
Working on : ATEN
2020-01-06   2020-12-31
Working on : ATGE
2020-01-06   2020-12-31
Working on : ATHX
2020-01-06   2020-12-31
Working on : ATI
2020-01-06   2020-12-31
Working on : ATKR
2020-01-06   2020-12-31
Working on : ATLC
2020-01-06   2020-12-31
Working on : ATLO
2020-01-06   2020-12-31
Working on : ATNI
2020-01-06   2020-12-31
Working on : ATNM
2020-01-06   2020-12-31
Working on : ATNX
2020-01-06   2020-12-31
Working on : ATO
2020-01-06   2020-12-31
Working on : 

Working on : CC
2020-01-06   2020-12-31
Working on : CCBG
2020-01-06   2020-12-31
Working on : CCF
2020-01-06   2020-12-31
Working on : CCI
2020-01-06   2020-12-31
Working on : CCK
2020-01-06   2020-12-31
Working on : CCL
2020-01-06   2020-12-31
Working on : CCMP
2020-01-06   2020-12-31
Working on : CCNE
2020-01-06   2020-12-31
Working on : CCO
2020-01-06   2020-12-31
Working on : CCOI
2020-01-06   2020-12-31
Working on : CCRN
2020-01-06   2020-12-31
Working on : CCS
2020-01-06   2020-12-31
Working on : CCXI
2020-01-06   2020-12-31
Working on : CDAY
2020-01-06   2020-12-31
Working on : CDE
2020-01-06   2020-12-31
Working on : CDEV
2020-01-06   2020-12-31
Working on : CDK
2020-01-06   2020-12-31
Working on : CDLX
2020-01-06   2020-12-31
Working on : CDMO
2020-01-06   2020-12-31
Working on : CDNA
2020-01-06   2020-12-31
Working on : CDNS
2020-01-06   2020-12-31
Working on : CDR
2020-01-06   2020-12-31
Working on : CDTX
2020-01-06   2020-12-31
Working on : CDW
2020-01-06   2020-12-31
Work

Working on : CYCC
2020-01-06   2020-12-31
Working on : CYH
2020-01-06   2020-12-31
Working on : CYRX
2020-01-06   2020-12-31
Working on : CYTK
2020-01-06   2020-12-31
Working on : CYTR
2020-01-06   2020-12-31
Working on : CZNC
2020-01-06   2020-12-31
Working on : CZR
2020-01-06   2020-12-31
Working on : CZWI
2020-01-06   2020-12-31
Working on : D
2020-01-06   2020-12-31
Working on : DAIO
2020-01-06   2020-12-31
Working on : DAKT
2020-01-06   2020-12-31
Working on : DAL
2020-01-06   2020-12-31
Working on : DAN
2020-01-06   2020-12-31
Working on : DAR
2020-01-06   2020-12-31
Working on : DARE
2020-01-06   2020-12-31
Working on : DBD
2020-01-06   2020-12-31
Working on : DBI
2020-01-06   2020-12-31
Working on : DBX
2020-01-06   2020-12-31
Working on : DCI
2020-01-06   2020-12-31
Working on : DCO
2020-01-06   2020-12-31
Working on : DCOM
2020-01-06   2020-12-31
Working on : DCPH
2020-01-06   2020-12-31
Working on : DD
2020-01-06   2020-12-31
Working on : DDD
2020-01-06   2020-12-31
Working 

2020-01-06   2020-12-31
Working on : FCBC
2020-01-06   2020-12-31
Working on : FCCO
2020-01-06   2020-12-31
Working on : FCEL
2020-01-06   2020-12-31
Working on : FCF
2020-01-06   2020-12-31
Working on : FCFS
2020-01-06   2020-12-31
Working on : FCN
2020-01-06   2020-12-31
Working on : FCNCA
2020-01-06   2020-12-31
Working on : FCPT
2020-01-06   2020-12-31
Working on : FCX
2020-01-06   2020-12-31
Working on : FDBC
2020-01-06   2020-12-31
Working on : FDP
2020-01-06   2020-12-31
Working on : FDS
2020-01-06   2020-12-31
Working on : FDX
2020-01-06   2020-12-31
Working on : FE
2020-01-06   2020-12-31
Working on : FEIM
2020-01-06   2020-12-31
Working on : FELE
2020-01-06   2020-12-31
Working on : FET
2020-01-06   2020-12-31
Working on : FF
2020-01-06   2020-12-31
Working on : FFBC
2020-01-06   2020-12-31
Working on : FFIC
2020-01-06   2020-12-31
Working on : FFIN
2020-01-06   2020-12-31
Working on : FFIV
2020-01-06   2020-12-31
Working on : FFNW
2020-01-06   2020-12-31
Working on : FFWM
20

...

2020-01-06   2020-12-31
Working on : ORI
2020-01-06   2020-12-31
Working on : ORLY
2020-01-06   2020-12-31
Working on : ORN
2020-01-06   2020-12-31
Working on : ORRF
2020-01-06   2020-12-31
Working on : OSBC
2020-01-06   2020-12-31
Working on : OSG
2020-01-06   2020-12-31
Working on : OSIS
2020-01-06   2020-12-31
Working on : OSK
2020-01-06   2020-12-31
Working on : OSPN
2020-01-06   2020-12-31
Working on : OSTK
2020-01-06   2020-12-31
Working on : OSUR
2020-01-06   2020-12-31
Working on : OTIC
2020-01-06   2020-12-31
Working on : OTTR
2020-01-06   2020-12-31
Working on : OUT
2020-01-06   2020-12-31
Working on : OVBC
2020-01-06   2020-12-31
Working on : OVLY
2020-01-06   2020-12-31
Working on : OXM
2020-01-06   2020-12-31
Working on : OXY
2020-01-06   2020-12-31
Working on : OZK
2020-01-06   2020-12-31
Working on : PACB
2020-01-06   2020-12-31
Working on : PACW
2020-01-06   2020-12-31
Working on : PAG
2020-01-06   2020-12-31
Working on : PAHC
2020-01-06   2020-12-31
Working on : PANL
2

Working on : RGCO
2020-01-06   2020-12-31
Working on : RGEN
2020-01-06   2020-12-31
Working on : RGLD
2020-01-06   2020-12-31
Working on : RGLS
2020-01-06   2020-12-31
Working on : RGNX
2020-01-06   2020-12-31
Working on : RGR
2020-01-06   2020-12-31
Working on : RGS
2020-01-06   2020-12-31
Working on : RH
2020-01-06   2020-12-31
Working on : RHE
2020-01-06   2020-12-31
Working on : RHI
2020-01-06   2020-12-31
Working on : RHP
2020-01-06   2020-12-31
Working on : RIBT
2020-01-06   2020-12-31
Working on : RICK
2020-01-06   2020-12-31
Working on : RIG
2020-01-06   2020-12-31
Working on : RIGL
2020-01-06   2020-12-31
Working on : RILY
2020-01-06   2020-12-31
Working on : RIOT
2020-01-06   2020-12-31
Working on : RJF
2020-01-06   2020-12-31
Working on : RKDA
2020-01-06   2020-12-31
Working on : RL
2020-01-06   2020-12-31
Working on : RLGT
2020-01-06   2020-12-31
Working on : RLGY
2020-01-06   2020-12-31
Working on : RLI
2020-01-06   2020-12-31
Working on : RLJ
2020-01-06   2020-12-31
Worki

Working on : SRT
2020-01-06   2020-12-31
Working on : SRTS
2020-01-06   2020-12-31
Working on : SSB
2020-01-06   2020-12-31
Working on : SSBI
2020-01-06   2020-12-31
Working on : SSD
2020-01-06   2020-12-31
Working on : SSKN
2020-01-06   2020-12-31
Working on : SSNC
2020-01-06   2020-12-31
Working on : SSP
2020-01-06   2020-12-31
Working on : SSTI
2020-01-06   2020-12-31
Working on : SSTK
2020-01-06   2020-12-31
Working on : ST
2020-01-06   2020-12-31
Working on : STAA
2020-01-06   2020-12-31
Working on : STAG
2020-01-06   2020-12-31
Working on : STAR
2020-01-06   2020-12-31
Working on : STBA
2020-01-06   2020-12-31
Working on : STC
2020-01-06   2020-12-31
Working on : STCN
2020-01-06   2020-12-31
Working on : STE
2020-01-06   2020-12-31
Working on : STIM
2020-01-06   2020-12-31
Working on : STKS
2020-01-06   2020-12-31
Working on : STLD
2020-01-06   2020-12-31
Working on : STOR
2020-01-06   2020-12-31
Working on : STRA
2020-01-06   2020-12-31
Working on : STRL
2020-01-06   2020-12-31


Working on : UNP
2020-01-06   2020-12-31
Working on : UNTY
2020-01-06   2020-12-31
Working on : UNVR
2020-01-06   2020-12-31
Working on : UONE
2020-01-06   2020-12-31
Working on : UONEK
2020-01-06   2020-12-31
Working on : UPLD
2020-01-06   2020-12-31
Working on : UPS
2020-01-06   2020-12-31
Working on : UPWK
2020-01-06   2020-12-31
Working on : URBN
2020-01-06   2020-12-31
Working on : URI
2020-01-06   2020-12-31
Working on : USAK
2020-01-06   2020-12-31
Working on : USAP
2020-01-06   2020-12-31
Working on : USB
2020-01-06   2020-12-31
Working on : USEG
2020-01-06   2020-12-31
Working on : USFD
2020-01-06   2020-12-31
Working on : USIO
2020-01-06   2020-12-31
Working on : USLM
2020-01-06   2020-12-31
Working on : USM
2020-01-06   2020-12-31
Working on : USNA
2020-01-06   2020-12-31
Working on : USPH
2020-01-06   2020-12-31
Working on : USWS
2020-01-06   2020-12-31
Working on : USX
2020-01-06   2020-12-31
Working on : UTHR
2020-01-06   2020-12-31
Working on : UTI
2020-01-06   2020-12-3

2020-01-06   2020-12-31
Working on : ZG
2020-01-06   2020-12-31
Working on : ZION
2020-01-06   2020-12-31
Working on : ZM
2020-01-06   2020-12-31
Working on : ZNGA
2020-01-06   2020-12-31
Working on : ZS
2020-01-06   2020-12-31
Working on : ZSAN
2020-01-06   2020-12-31
Working on : ZTS
2020-01-06   2020-12-31
Working on : ZUMZ
2020-01-06   2020-12-31
Working on : ZUO
2020-01-06   2020-12-31
Working on : ZVO
2020-01-06   2020-12-31
Working on : ZYNE
2020-01-06   2020-12-31
Working on : ZYXI
2020-01-06   2020-12-31


In [21]:
# Get top 20 stocks ROI
market_df.sort_values(by=['ROI'], ascending=False).head(20)

Ticker       COV        ROI
1851   NVAX  0.713963  23.347162
2753   VXRT  0.696748  15.794118
1444   KIRK  0.979047  13.078740
2193   RIOT  0.931865  12.069231
514    CELH  0.714738  10.130531
1603   MARA  1.101630   9.762886
2700   VERI  0.740303   9.576208
1878    OAS  2.341170   9.237569
1805    NLS  0.668594   8.547368
2042   PLUG  0.767566   7.876963
2682    UUU  0.690005   7.491526
2583   TSLA  0.576388   6.814036
200    APPS  0.795809   6.663957
1628   MCRB  0.878020   6.469512
1752   MVIS  0.844106   6.369863
2825   WTRH  0.581070   6.315789
2238    RVP  0.585767   5.929032
566    CLDX  0.715105   5.712644
2606   TWST  0.596517   5.324530
2791   WKHS  0.761858   5.161994

**Merge Multiple Stocks in One Dataframe by Column Name**

In [ ]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # Use a mask to grab data between defined dates
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

**Create a Correlation Matrix using FAANGS**

In [ ]:
# Correlation tells us how closely 2 stocks returns move together
# Correlation is a standardized value lying between -1 and 1
# When this value is greater that .5 we say that these stocks are strongly correlated
# Of course each stocks price is perfectly correlated with itself

# We focus on the correlation of returns because investors care about returns 

# Merge all stock price data into 1 dataframe
faang_list = ["FB", "AMZN", "AAPL", "NFLX", "GOOG"]
mult_df = merge_df_by_column_name('daily_return',  '2020-1-1', '2020-12-31', *faang_list)
mult_df

# Generate a Correlation Matrix
mult_df.corr()

# We can look at the correlation between Netflix and the others
mult_df.corr()['NFLX']

# We can plot this in a bar chart
mult_df.corr()['NFLX'].plot(kind='bar');

**Get the Variance of a Stock**

Covariance is the measure of the relationship between 2 blocks of data. The covariance of a stock to itself is the variance of that variable.

In [ ]:
mult_df.cov() * 253

**Why do We Care About Risk**

Most investors don't handle massive flucuations in stock prices well. What we want to do at the very least is to make them aware of how dramatically their portfolios returns may be. We can then do our best to minimize risk by adding other stocks that have returns that aren't as closely correlated.

**Calculating a Portfolios Variance**

When calculating the variance of a portfolio we must define its weight, or how much of the portfolio it makes up. If you add up the weight of all stocks you get a value of 1.

w1, w2 = Stock Weights

sd1, sd2 = Stock Standard Deviations

Portfolio Variance = 

Since 

Then the Portfolio Variance = 


In [ ]:
# Let's create a portfolio made up of Facebook & Newmont Corporation
port_list = ["FB", "NEM"]
port_df = merge_df_by_column_name('daily_return',  '2020-1-1', '2020-12-31', *port_list)
port_df

# Generate a Correlation Matrix to see that they are not correlated
port_df.corr()

# Get prices at the beginning of Jan 2020 to calculate weight of both stocks
price_df = merge_df_by_column_name('Adj Close',  '2020-1-1', '2020-12-31', *["FB","NEM"])
price_df.head()

# I'll buy 1 share of FB at 209.78 & 5 of NEM for 41.74 for a total of 208.70
# Total Value = $418.48
fb_wt = 209.78 / 418.48
nem_wt = 208.70 / 418.48
fb_wt = .5012 # .5012
nem_wt = .4988 # .4987 and we'll change it to .4988 so they equal 1
wts = np.array([fb_wt, nem_wt])

# The portfolio variance is found from matrix multiplication
# We transpose the weights, multiply by the covarience and then the weight to get 
# the covariance of the portfolio
wts.T
port_var = np.dot(wts.T, np.dot(port_df.cov() * 253, wts))
print("Portfolio Var :", port_var)
print("FB Var :", port_df["FB"].var() * 253)
print("NEM Var :", port_df["NEM"].var() * 253)